In [2]:
import pandas as pd

In [7]:
df = pd.read_csv('../../public/toilets_san_diego_ca.csv')
print(df.isnull().sum())

id                      0
site                    1
address                14
latitude              363
longitude             363
hours                   7
cleaning_frequency     46
public                363
content               363
images                363
operated_by            28
author_id             360
created_at            363
updated_at            363
dtype: int64


In [11]:
df = df.fillna('Unknown')
df.to_csv('../../public/toilets_san_diego_ca_cleaned.csv', index=False)
df = pd.read_csv('../../public/toilets_san_diego_ca_cleaned.csv')
print(df.isnull().sum())

id                    0
site                  0
address               0
latitude              0
longitude             0
hours                 0
cleaning_frequency    0
public                0
content               0
images                0
operated_by           0
author_id             0
created_at            0
updated_at            0
dtype: int64


In [15]:
df = pd.read_csv('../../public/toilets_san_diego_ca_cleaned.csv')
df = df.drop(columns=['id'])
df.to_csv('../../public/toilets_san_diego_ca_cleaned.csv', index=False)

In [17]:
df = pd.read_csv('../../public/toilets_san_diego_ca_cleaned.csv')
# change columns latitude and longitude into numeric types
df['latitude'] = 0
df['longitude'] = 0
df.to_csv('../../public/toilets_san_diego_ca_cleaned.csv', index=False)

In [21]:
from datetime import datetime

# Get the current date and time in UTC - Unix Timestamp
current_timestamp = datetime.timestamp(datetime.now())

df = pd.read_csv('../../public/toilets_san_diego_ca_cleaned.csv')
df['public'] = True
df['created_at'] = current_timestamp
df['updated_at'] = current_timestamp
df.to_csv('../../public/toilets_san_diego_ca_cleaned.csv', index=False)

In [41]:
df = pd.read_csv('../../public/toilets_san_diego_ca_cleaned.csv')
df['images'] = {}
df.to_csv('../../public/toilets_san_diego_ca_cleaned.csv')
df = df[['site', 'address', 'latitude', 'longitude', 'hours', 'cleaning_frequency', 'public', 'content', 'images', 'operated_by', 'author_id', 'created_at', 'updated_at']]
# Assign an empty array to each entry in the 'images' column
# `{}` is the syntax for an array in PostgreSQL
df['images'] = [[] for _ in range(len(df))]
df.to_csv('../../public/toilets_san_diego_ca_cleaned.csv', index=False)

In [52]:
df = pd.read_csv('../../public/toilets_san_diego_ca_cleaned.csv')
df['author_id']

0      Unknown
1      Unknown
2      Unknown
3      Unknown
4      Unknown
        ...   
358    Unknown
359    Unknown
360    Unknown
361    Unknown
362    Unknown
Name: author_id, Length: 363, dtype: object

In [54]:
import psycopg2
conn = psycopg2.connect("dbname=toilets_fyi user=postgres")
cur = conn.cursor()

df = pd.read_csv('../../public/toilets_san_diego_ca_cleaned.csv')
df['images'] = "{}"

try:
    for index, row in df.iterrows():
        cur.execute("INSERT INTO toilets (site, address, latitude, longitude, hours, cleaning_frequency, public, content, images, operated_by, created_at, updated_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, to_timestamp(%s), to_timestamp(%s))", 
                    (row['site'], row['address'], row['latitude'], row['longitude'], row['hours'], row['cleaning_frequency'], row['public'], row['content'], row['images'], row['operated_by'], row['created_at'], row['updated_at']))

except Exception as e:
    print(e)
    conn.rollback()

conn.commit()
cur.close()
conn.close()


In [56]:
conn = psycopg2.connect("dbname=toilets_fyi user=postgres")
cur = conn.cursor()
cur.execute("SELECT * FROM toilets")
print(len(cur.fetchall()))
conn.close()

363
